# Pareto/NBD Model
The Pareto/Negative-Binomial Distribution model was the first Buy-Till-You-Die (BTYD) model for estimating non-contractual customer activity over a continuous time period. First introduced by Schmittlein, et. al. in 1987 and developed further by Bruce Hardie and Peter Fader, it is frequently used as a benchmark in CLV research due to its robust performance and wide range of functionality. For detailed derivations of this model please refer to
["A Note on Deriving the Pareto/NBD Model and Related Expressions."](https://www.brucehardie.com/notes/009/pareto_nbd_derivations_2005-11-05.pdf)

In this notebook we will use Bayesian inference to fit a Pareto/NBD model in PyMC-Marketing, and compare results with the frequentist [`lifetimes`](https://github.com/CamDavidsonPilon/lifetimes) library (no longer maintained). We will also demonstrate the predictive functionality of this model, along with an example for time-invariant covariates.

## Setup Notebook

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sb
import xarray as xr
from fastprogress.fastprogress import progress_bar
from lifetimes import ParetoNBDFitter

from pymc_marketing import clv

# Plotting configuration
az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

## Load Data

In this notebook we will be using the CDNOW sample dataset, a popular benchmarking dataset in CLV and BTYD modeling research. Refer [here](https://www.brucehardie.com/notes/026/) for more information about the dataset.

In [ ]:
url_cdnow = "https://raw.githubusercontent.com/pymc-labs/pymc-marketing/main/data/cdnow_transactions.csv"

raw_data = pd.read_csv(url_cdnow)

raw_data.info()
raw_data.head()

The only requirements for modeling spending behaviour with `ParetoNBDModel` are a customer identifier column, and a datetime column for each purchase. The number of CDs purchased and money spent per transaction could also be useful covariates, so we'll keep them in mind for later.

It is common for customer transaction databases to also contain returns, discount values, etc., so let's do a quick validation check:

In [ ]:
raw_data.describe()

Note there were some transactions with spend values of 0! Perhaps these were returns or promotional giveaways. Instances such as this are not true purchasing activities, and should be excluded from modeling.

In [ ]:
raw_data = raw_data[raw_data["spent"] > 0]

Use the `rfm_summary` utility to aggregate data for modeling:

In [ ]:
rfm_data = clv.rfm_summary(
    raw_data,
    customer_id_col="id",
    datetime_col="date",
    datetime_format="%Y%m%d",
    time_unit="W",
)

rfm_data.info()
rfm_data.head()

Recall the data aggregation definitions from the [CLV Quickstart](https://www.pymc-marketing.io/en/stable/notebooks/clv/clv_quickstart.html):

- `customer_id` is an index of a unique identifiers for each customer.
- `frequency` is the number of repeat purchases that a customer has made (i.e., total number of purchases minus one).
- `recency` indicates the time period when a customer made their most recent purchase. If a customer has only made 1 purchase, recency is 0.
- `T` is a customer's "age", or the number of time periods since their first purchase.

## Model Definition
The Pareto/NBD model is based on the following assumptions for each customer:
1. Customers are active for an unobserved period of time, then become permanently inactive.
   
#### Purchasing Process

2. While active, the the number of transactions made by a customer follows a Poisson process with transaction rate $\lambda$:
   
   $$P(X(t)=x|\lambda) = \frac{(\lambda t)^{x}e^{-\lambda t}}{x!}, x=0,1,2,...$$
   
   This is equivalent to assuming time between transactions is exponentially distributed with transaction rate $\lambda$:
   
   $$f(t_{j}-t_{j-1}| \lambda) = \lambda e^{-\lambda (t_{j} - t_{j - 1})}, \quad t_{j} \geq t_{j - 1} \geq 0$$
   
   Where $t$ is the time period of the $j$th purchase.
3. Heterogeneity in $\lambda$ follows a Gamma distribution with shape parameter $r$ and scale parameter $\alpha$:

    $$g(\lambda|r, \alpha) = \frac{\alpha^{r}\lambda^{r - 1}e^{-\lambda \alpha}}{\Gamma(r)}$$
#### Dropout Process
4. The duration of a customer's unobserved active lifetime is exponentially distributed with dropout rate $\mu$.

5. Heterogeneity in $\mu$ also follows a Gamma distribution with shape parameter $s$ and scale parameter $\beta$:

    $$g(\mu|s, \beta) = \frac{\beta^{s}\mu^{s - 1}e^{-\mu \beta}}{\Gamma(s)}$$
   
6. Transaction rate $\lambda$ and time until dropout $\mu$ vary independently for each customer.

If we take the expectation across the distributions of $\lambda$ and $\mu$, we can derive a likelihood function to estimate parameters $r$, $\alpha$, $s$, and $\beta$ across the customer population. For more details on the `ParetoNBD` likelihood please refer to the [docs](https://www.pymc-marketing.io/en/stable/api/generated/pymc_marketing.clv.distributions.ParetoNBD.html#pymc_marketing.clv.distributions.ParetoNBD).

## Model Fitting

### `lifetimes` Benchmark Model

Let's time travel back to July 2020 and use the old `lifetimes` library to fit a Pareto/NBD model with Maximum Likelihood Estimation (MLE). The `Nelder-Mead` optimizer from `scipy.optimize` is ran under the hood to estimate scalar values for $r$, $\alpha$, $s$, and $\beta$.

In [ ]:
freq = rfm_data["frequency"].values
rec = rfm_data["recency"].values
T = rfm_data["T"].values

pnbd_lt = ParetoNBDFitter()
pnbd_lt.fit(freq, rec, T)
pnbd_lt.params_.sort_index()

The Bayesian equivalent of MLE is Maximum a Posteriori(MAP), in which the returned scalar values are regularized with priors during estimation.

A "Flat" prior indicates the user is agnostic, holding no prior beliefs or assumptions about the data. $r$, $\alpha$, $s$, and $\beta$ must also be positive values, so let's configure our Bayesian `ParetoNBDModel` with `HalfFlat` priors:

In [ ]:
flat_config = {
    "r_prior": {"dist": "HalfFlat", "kwargs": {}},
    "alpha_prior": {"dist": "HalfFlat", "kwargs": {}},
    "s_prior": {"dist": "HalfFlat", "kwargs": {}},
    "beta_prior": {"dist": "HalfFlat", "kwargs": {}},
}

pnbd_pymc = clv.ParetoNBDModel(data=rfm_data, model_config=flat_config)

Build the model to view the choice of Priors used for modeling:

In [ ]:
pnbd_pymc.build_model()  # optional step
pnbd_pymc

Note it is not necessary to build a model prior to modeling.

Now let's fit our `ParetoNBDModel` with MAP.

In [ ]:
idata_map = pnbd_pymc.fit(fit_method="map")

For MAP fitting `pymc-marketing` uses the `L_BGFS-B` optimizer from `scipy.optimize`, a faster and more stable alternative to `Nelder-Mead`.

In [ ]:
flat_fit = pnbd_pymc.fit_summary()
print(flat_fit)

Model parameter estimations are equivalent between the frequentist MLE fit from `lifetimes`, and a Bayesian `pymc-marketing` model fit with flat priors. However, the CDNOW sample we're working with is quite small and comprises only 10% of the total CDNOW dataset, so it's quite likely these estimates are overfitting if we attempt to run predictions on the full dataset.

With prior distributions, we can inform model fitting with our own subjective domain knowledge, and even improve the speed of model fits. The default prior configuration for `ParetoNBDModel` works well for a variety of use cases:

In [ ]:
pnbd_map = clv.ParetoNBDModel(data=rfm_data)
pnbd_map.build_model()  # required for prior predictive checks
pnbd_map

#### Prior and Posterior Predictive Checks
PPCs allow us to check the efficacy of our priors, and the peformance of the fitted posteriors. PPCs aren't usually an option with MAP fitted models, but here we're actually sampling from the latent $\lambda$ and $\mu$ Gamma distributions, so PPCs are possible for `ParetoNBDModel` regardless of the fit method!

Let's see how the model performs in a *prior* predictive check, where we sample from the default priors before fitting the model: 

In [ ]:
with pnbd_map.model:
    prior_idata = pm.sample_prior_predictive(random_seed=45, samples=1)

obs_freq = prior_idata.observed_data["recency_frequency"].sel(obs_var="frequency")
ppc_freq = prior_idata.prior_predictive["recency_frequency"].sel(obs_var="frequency")[
    0
][0]

# PPC histogram plot
pd.DataFrame(
    {
        "Model Estimations": ppc_freq.to_pandas().value_counts().sort_index(),
        "Observed": obs_freq.to_pandas().value_counts().sort_index(),
    }
).head(15).plot(
    kind="bar",
    title="Prior Predictive Check of Repeat Purchases per Customer",
    xlabel="Repeat Purchases per Customer",
    ylabel="Number of Customers",
    figsize=(12, 7),
);

In [ ]:
# create axes for each companion plot
_, axes = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(12, 7),
)
axes = axes.flatten()

# ArviZ ECDF plot
az.plot_ecdf(
    ppc_freq,
    obs_freq,
    confidence_bands=True,
    ax=axes[0],
    fill_kwargs={"label": "95% HDI"},
    plot_kwargs={"label": "Observed"},
).set(
    title="Prior Predictive ECDF Plot",
    xlabel="Purchases per Customer",
    ylabel="Percent of Total",
)
axes[0].legend(loc="lower right")

# ArviZ ECDF Diff plot
az.plot_ecdf(
    ppc_freq,
    obs_freq,
    confidence_bands=True,
    difference=True,
    ax=axes[1],
    fill_kwargs={"label": "95% HDI"},
    plot_kwargs={"label": "Observed"},
).set(
    title="Prior Predictive Difference Plot",
    xlabel="Purchases per Customer",
    ylabel="Percent Deviation from Expectation",
)
axes[1].legend(loc="upper right");

Here the `ParetoNBDModel` is simulating customer purchases from the prior distributions to compare against the obseved data. The priors we specified look reasonable for customers who have made 1-6 purchases, but does not model single-purchase customers well. The deviations beyond the confidence intervals of the Emperical Cumulative Distribution Function (ECDF) and Difference plots indicate the importance of single purchases customers - in fact they comprise over half the CDNOW sample dataset! 

Let's fit our model and run a *posterior* predictive check for comparison:

In [ ]:
pnbd_map.fit()
map_fit = pnbd_map.fit_summary()  # save for plotting later

obs_freq = pnbd_map.idata.observed_data["recency_frequency"].sel(obs_var="frequency")
ppc_freq = pnbd_map.distribution_new_customer_recency_frequency(
    rfm_data,
    random_seed=42,
).sel(chain=0, draw=0, obs_var="frequency")

# PPC histogram plot
pd.DataFrame(
    {
        "Model Estimations": ppc_freq.to_pandas().value_counts().sort_index(),
        "Observed": obs_freq.to_pandas().value_counts().sort_index(),
    }
).head(15).plot(
    kind="bar",
    title="Posterior Predictive Check of Repeat Purchases per Customer",
    xlabel="Repeat Purchases per Customer",
    ylabel="Number of Customers",
    figsize=(12, 7),
);

In [ ]:
# create axes for each companion plot
_, axes = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(12, 7),
)
axes = axes.flatten()

# ArviZ ECDF plot
az.plot_ecdf(
    ppc_freq,
    obs_freq,
    confidence_bands=True,
    ax=axes[0],
    fill_kwargs={"label": "95% HDI"},
    plot_kwargs={"label": "Observed"},
).set(
    title="Posterior Predictive ECDF Plot",
    xlabel="Purchases per Customer",
    ylabel="Percent of Total",
)
axes[0].legend(loc="lower right")

# ArviZ ECDF Diff plot
az.plot_ecdf(
    ppc_freq,
    obs_freq,
    confidence_bands=True,
    difference=True,
    ax=axes[1],
    fill_kwargs={"label": "95% HDI"},
    plot_kwargs={"label": "Observed"},
).set(
    title="Posterior Predictive Difference Plot",
    xlabel="Purchases per Customer",
    ylabel="Percent Deviation from Expectation",
)
axes[1].legend(loc="upper right");

Our fitted model is able to reliably simulate customer behavior!

## Full Bayesian Inference

MAP fits estimate only scalar values for $r$, $\alpha$, $s$, and $\beta$, but with full Bayesian sampling we can infer the posterior probability distributions for these parameters, illustrating uncertainty in our estimates as well as enabling prediction intervals.

NUTS is the default sampler in `pymc-marketing`, which samples from the posterior by exploring the gradients of the probability space. However, NUTS sampling with `ParetoNBDModel` can be quite slow due to the complexity of the likelihood expression. In fact, the mathematical complexity of this model is what motivated the development of the [`BetaGeoModel`](https://www.pymc-marketing.io/en/stable/notebooks/clv/bg_nbd.html) in 2005. The BG/NBD model makes some simplifying assumptions and sacrifices functionality in customer dropout estimation for better computational performance.

To save time and computational costs, it is recommended to use gradient-free methods such as `pm.DEMetropolisZ`. This often requires more samples during fitting, so if any `rhat statistic` warnings are encountered, increase the size of the `tune` and `draw` parameters until the warning no longer appears.

In [ ]:
pnbd_full = clv.ParetoNBDModel(data=rfm_data)
pnbd_full.build_model()

with pnbd_full.model:
    pnbd_full.idata = pm.sample(
        step=pm.DEMetropolisZ(),
        tune=2500,
        draws=3000,
        idata_kwargs={"log_likelihood": True},
    )

In [ ]:
pnbd_full.fit_summary()

In [ ]:
axes = az.plot_trace(
    data=pnbd_full.idata,
    compact=True,
    kind="rank_bars",
    backend_kwargs={"figsize": (12, 7), "layout": "constrained"},
)
plt.gcf().suptitle("Pareto/NBD Model Trace", fontsize=18, fontweight="bold");

In [ ]:
pnbd_full.idata

Let's see how the DEMZ posteriors compare to the MAP estimations:

In [ ]:
_, axes = plt.subplots(
    nrows=2, ncols=2, figsize=(12, 7), sharex=False, sharey=False, layout="constrained"
)

axes = axes.flatten()

for i, var_name in enumerate(["r", "alpha", "s", "beta"]):
    ax = axes[i]
    az.plot_posterior(
        pnbd_full.idata.posterior[var_name].values.flatten(),
        color="C0",
        point_estimate="mean",
        ax=ax,
        label="DEMZ",
    )
    ax.axvline(x=map_fit[var_name], color="C1", linestyle="--", label="MAP")
    ax.legend(loc="upper right")
    ax.set_title(var_name)

plt.gcf().suptitle("Pareto/NBD Model Parameters", fontsize=18, fontweight="bold");

After fitting, models can be persisted for later use:

In [ ]:
pnbd_pymc.save("pnbd.nc")
pnbd_pymc.load("pnbd.nc")

## Predictive Methods

The Pareto/NBD model supports a variety of predictive methods:

- `expected_purchases`
- `expected_probability_alive`
- `expected_purchases_new_customer`
- `expected_purchase_probability`

Let's take a small sample of users:

In [ ]:
example_customer_ids = [1, 5, 10, 18, 46, 1413]

rfm_sample = rfm_data.query("customer_id.isin(@example_customer_ids)")

rfm_sample.sort_values(by="frequency")

Observe customers 5 & 10 are *non-repeat buyers*, whereas 1413 and 46 are *frequent buyers*.

### Expected Number of Purchases
Let's plot each customer's expected number of purchases over the next $90$ time periods:

In [ ]:
time_periods = 90

expected_purchases_over_time = xr.concat(
    objs=[
        pnbd_full.expected_purchases(
            data=rfm_sample,
            future_t=t,
        )
        for t in progress_bar(range(time_periods))
    ],
    dim="t",
).transpose(..., "t")

In [ ]:
_, axes = plt.subplots(
    nrows=len(example_customer_ids),
    ncols=1,
    figsize=(12, 14),
    sharex=True,
    sharey=True,
    layout="constrained",
)

axes = axes.flatten()

for i, customer_id in enumerate(example_customer_ids):
    ax = axes[i]
    customer_expected_purchases = expected_purchases_over_time.sel(
        customer_id=customer_id
    )
    az.plot_hdi(
        range(time_periods),
        customer_expected_purchases,
        hdi_prob=0.94,
        color="C0",
        fill_kwargs={"alpha": 0.3, "label": "$94 \\%$ HDI"},
        ax=ax,
    )
    az.plot_hdi(
        range(time_periods),
        customer_expected_purchases,
        hdi_prob=0.5,
        color="C0",
        fill_kwargs={"alpha": 0.5, "label": "$50 \\%$ HDI"},
        ax=ax,
    )
    ax.plot(
        range(time_periods),
        customer_expected_purchases.mean(dim=("chain", "draw")),
        color="C0",
        label="posterior mean",
    )
    ax.legend(loc="upper left")
    ax.set(title=f"Customer {customer_id}", xlabel="t", ylabel="# of purchases")

axes[-1].set(xlabel="t")
plt.gcf().suptitle("Expected Number of Purchases", fontsize=18, fontweight="bold");

Note the HDI prediction intervals are only available if the model is fit with full posteriors.

Observe the large number of purchases expected from the frequent buyers (Customers 46 and 1413) whereas little or no future activity is expected of the remaining customers.

### Expected Probability Alive
Let's check the probability our customers are still alive and compare against the MAP results.

In [ ]:
_, axes = plt.subplots(
    nrows=3,
    ncols=2,
    figsize=(12, 14),
    layout="constrained",
)

axes = axes.flatten()

for i, customer_id in enumerate(example_customer_ids):
    ax = axes[i]
    demz_alive = pnbd_full.expected_probability_alive(
        rfm_sample,
        future_t=0,
    ).sel(customer_id=customer_id)
    map_alive = pnbd_map.expected_probability_alive(rfm_sample, future_t=0).sel(
        customer_id=customer_id
    )

    az.plot_density(demz_alive, hdi_prob=1, colors="C1", shade=0.3, bw=0.005, ax=ax)
    ax.axvline(x=map_alive, color="C3", linestyle="--", label="MAP")
    ax.legend(loc="upper right")
    ax.set(
        title=f"Customer {customer_id}",
        xlabel="Probability Alive",
        ylabel="Posterior Density",
    )

plt.gcf().suptitle("Expected Probability Alive", fontsize=18, fontweight="bold");

Customer 1413 has a rather low alive probability despite being a frequent purchaser. This would be a good example of a customer to target with a special offer for retention.

These probabilities are estimated at time period 0, but we can also estimate the probabilities customers will still be active in the future. Let's calculate the posterior densities 90 time periods from now and compare to the MAP at time period 0:

In [ ]:
_, axes = plt.subplots(
    nrows=3,
    ncols=2,
    figsize=(12, 14),
    layout="constrained",
)

axes = axes.flatten()

for i, customer_id in enumerate(example_customer_ids):
    ax = axes[i]
    demz_alive = pnbd_full.expected_probability_alive(
        rfm_sample,
        future_t=90,
    ).sel(customer_id=customer_id)
    map_alive = pnbd_map.expected_probability_alive(rfm_sample, future_t=0).sel(
        customer_id=customer_id
    )

    az.plot_density(demz_alive, hdi_prob=1, colors="C1", shade=0.3, bw=0.005, ax=ax)
    ax.axvline(x=map_alive, color="C3", linestyle="--", label="MAP at Time 0")
    ax.legend(loc="upper left")
    ax.set(
        title=f"Customer {customer_id}",
        xlabel="Probability Alive",
        ylabel="Posterior Density at Time 90",
    )

plt.gcf().suptitle("Expected Probability Alive Delta", fontsize=18, fontweight="bold");

Pay attention to the x-axes for each customer - The probabilities barely changed for the *non-repeat* customers, but there is a significant delta for *frequent buyers*.

A good rule of thumb is that an alive probability of .25-.30 usually indicates an at-risk or inactive customer. Future projections can give additional insight into customer churn risk.

### Probability of $n$ Purchases over Time $t$

Customer 46 is our best customer in this small sample set, and is expected to make at least $15$ purchases over the next $90$ time periods. What is the probability of this many purchases being made, and how will it change over time?

Let's plot a heatmap to paint the full picture:

In [ ]:
# create arrays of parameter combinations
n_purchases = np.repeat([0, 3, 6, 9, 12, 15], 6)
time_periods = np.tile([15, 30, 45, 60, 75, 90], 6)

expected_purchase_prob_heatmap = xr.concat(
    objs=[
        pnbd_map.expected_purchase_probability(
            rfm_sample,
            n_purchases=params[0],
            future_t=params[1],
        ).sel(customer_id=46)
        for params in zip(n_purchases, time_periods, strict=False)
    ],
    dim="customer_id",
).transpose(..., "customer_id")

In [ ]:
heatmap_reshape = expected_purchase_prob_heatmap.values.reshape(6, 6)

sb.heatmap(heatmap_reshape, annot=True)

plt.xlabel("Time Periods")
plt.xticks(np.arange(6) + 0.5, [15, 30, 45, 60, 75, 90], rotation=0)
plt.ylabel("Number of Purchases")
plt.yticks(np.arange(6) + 0.5, [0, 3, 6, 9, 12, 15], rotation=0)
plt.gcf().suptitle(
    "Expected Purchase Probabilities for Customer 46", fontsize=18, fontweight="bold"
);

This heatmap highlights how Customer 46 is expected to make at least 15 purchases *up to* time period 90, but the odds of 15 purchases being made before time period 75 or even time period 60 are slightly higher. Also note these probabilities assume exact expectations (i.e., there's a 6.2% chance of the 15th purchase being made precisely during time period 60.)

### Expected Number of Purchases for New Customers
So far we've only been running predictions for existing customers, but we can also estimate the expected number of transactions over time for a new customer:

In [ ]:
expected_purchases_over_time_new_customer = xr.concat(
    objs=[
        pnbd_full.expected_purchases_new_customer(
            data=rfm_sample,
            t=t,
        ).sel(customer_id=1)  # customer_id is arbitrary here
        for t in range(90)
    ],
    dim="t",
).transpose(..., "t")


# plot results
ax = plt.axes()

az.plot_hdi(
    range(90),
    expected_purchases_over_time_new_customer,
    hdi_prob=0.94,
    color="C2",
    fill_kwargs={"alpha": 0.3, "label": "$94 \\%$ HDI"},
    ax=ax,
)
az.plot_hdi(
    range(90),
    expected_purchases_over_time_new_customer,
    hdi_prob=0.5,
    color="C2",
    fill_kwargs={"alpha": 0.5, "label": "$50 \\%$ HDI"},
    ax=ax,
)
ax.plot(
    range(90),
    expected_purchases_over_time_new_customer.mean(dim=("chain", "draw")),
    color="C2",
    label="posterior mean",
)
ax.legend(loc="upper left")
ax.set(
    title="Expected Number of Purchases by New Customers",
    ylabel="# of purchases",
    xlabel="t",
);

Let's see how these estimates change when we add covariates to the model.

## Time-Invariant Covariates

Recall $\alpha$, and $\beta$ represent the scale parameters for the purchase and dropout rate distributions, respectively. To model for time-invariant covariates, we simply modify these parameters as follows:

$$\alpha = \alpha_0e^{-\gamma_1'z_1}$$
$$\beta = \beta_0e^{-\gamma_2'z_2}$$

Where $\gamma_1$ and $\gamma_2$ are coefficients capturing the impact of the covariates, and $z_1$ and $z_2$ the covariate arrays for each customer.

Let's take a look at the covariates available in the raw data:

In [ ]:
# aggregate raw data by customer id
covar_df = raw_data[["id", "cds_bought", "spent"]].groupby("id").mean().reset_index()

# plot covariate histograms
_, axes = plt.subplots(
    nrows=1,
    ncols=2,
    figsize=(10, 5),
    layout="constrained",
)

axes = axes.flatten()

covars = ["cds_bought", "spent"]
colors = ["C0", "C2"]

for ax in zip(axes, covars, colors, strict=False):
    ax[0].hist(
        x=covar_df[ax[1]],
        bins=5,
        color=ax[2],
    )
    ax[0].set(title=f"{ax[1]}", xlabel="value", ylabel="count")

One-tailed distributions with large values like this will complicate model fitting, so let's log and standardize our covariates:

In [ ]:
for covar in ["cds_bought", "spent"]:
    covar_df[f"log_std_{covar}"] = np.log(covar_df[covar]).copy()
    covar_df[f"log_std_{covar}"] -= np.nanmean(covar_df[f"log_std_{covar}"])
    covar_df[f"log_std_{covar}"] /= np.nanstd(covar_df[f"log_std_{covar}"])
    covar_df[f"log_std_{covar}"] = covar_df[f"log_std_{covar}"].fillna(0)

_, axes = plt.subplots(
    nrows=1,
    ncols=2,
    figsize=(10, 5),
    layout="constrained",
)

axes = axes.flatten()

for ax in zip(axes, covars, colors, strict=False):
    ax[0].hist(
        x=covar_df[f"log_std_{ax[1]}"],
        bins=5,
        color=ax[2],
    )
    ax[0].set(title=f"transformed {ax[1]}", xlabel="value", ylabel="count")

To parametrize `ParetoNBDModel` with covariates, join the covariates to the existing RFM data and specify the column names in the `model_config`. Covariates for the purchase and dropout rates can be specified separately, enabling experimentation with various combinations to find what works best:

In [ ]:
rfm_covar = rfm_data.merge(covar_df, left_on="customer_id", right_on="id", how="inner")

pnbd_covar = clv.ParetoNBDModel(
    rfm_covar,
    model_config={
        "purchase_covariate_cols": ["log_std_cds_bought", "log_std_spent"],
        "dropout_covariate_cols": ["log_std_cds_bought", "log_std_spent"],
    },
)

pnbd_covar.build_model()
pnbd_covar

The additional parameters are automatically created when covariates are added.

Let's do a quick MAP fit and check the results:

In [ ]:
pnbd_covar.fit(fit_method="map")

print("Fitted Model Parameters")
summary = pnbd_covar.fit_summary(
    var_names=[
        "r",
        "alpha_scale",
        "s",
        "beta_scale",
        "purchase_coefficient",
        "dropout_coefficient",
    ]
)
print(summary)

The `purchase_coefficient` and `dropout_coefficient` parameters indicate the respective impacts of each covariate - a negative sign can be interpreted as "less likely to make a purchase or drop out".

We can see the average number of CDs per purchase only has a small impact on the time between purchases, but a rather large impact on the dropout rate. Customers who frequently purchase multiple CDs are the least likely to dropout.

The average spend per purchase is significant for both purchasing and time until dropout, but note that  if using the [Gamma-Gamma model](https://www.pymc-marketing.io/en/stable/notebooks/clv/gamma_gamma.html) to estimate customer lifetime value per the [Quickstart](https://www.pymc-marketing.io/en/stable/notebooks/clv/clv_quickstart.html), then *average spend cannot be used as a covariate* because an important modeling assumption for the Gamma-Gamma model is that spend and frequency are uncorrelated.

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -p pymc,pytensor